<a href="https://colab.research.google.com/github/Murcha1990/ML_Course_PT/blob/main/Lecture2_ClassicML/DGA_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm

In [3]:
!wget -O dga.csv "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/f2Z4w-xHheenKg"

--2026-01-26 12:33:58--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/f2Z4w-xHheenKg
Resolving getfile.dokpub.com (getfile.dokpub.com)... 138.199.175.196
Connecting to getfile.dokpub.com (getfile.dokpub.com)|138.199.175.196|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/43b744c0869ddba303ec995686e5796d8d7da922aec887f844dbcb27550746da/69779777/0F8ZdSEE6bEiq3zgKLsJP05XO4AC9ufyrXMW5UgtacVTptX9sWjOnpcHQpu7iCbA3x-wCfRFjW8x1tXXep-8yA%3D%3D?uid=0&filename=train_dga.csv&disposition=attachment&hash=ES%2BZaDNM863i5u9PjpBnP1sJhtbTnNSlZg%2Btb04JzVMo4tv58hzotGiOIsxkjVRPq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=text%2Fcsv&owner_uid=259847718&fsize=331518539&hid=2e72185b076ef407efbf7bedfed36b5f&media_type=data&tknv=v3 [following]
--2026-01-26 12:33:59--  https://downloader.disk.yandex.ru/disk/43b744c0869ddba303ec995686e5796d8d7da922aec887f844dbcb27550746da/69779777/0F8ZdSEE6bEiq3zgKLsJP05XO4AC9ufyrXMW5

In [4]:
import numpy as np
import pandas as pd

from pathlib import Path
from math import log2

from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [5]:
VOWELS = set("aeiou") # множество гласных букв

def shannon_entropy(s):
    probs = [s.count(c) / len(s) for c in set(s)]
    return -sum(p * log2(p) for p in probs)

def digit_ratio(name):
    length = len(name)
    digits = sum(c.isdigit() for c in name)
    return digits / max(1, length)

def letter_count(name):
    return sum(c.isalpha() for c in name)

def has_digits(name):
    return int(any(c.isdigit() for c in name))

def vowel_ratio(name):
    letters = sum(c.isalpha() for c in name)
    vowels = sum(c in VOWELS for c in name)
    return vowels / max(1, letters)

def extract_features(domain):
    name = domain.split('.')[0]

    length = len(name)                                      # длина
    digits = sum(c.isdigit() for c in name)                 # количество цифр
    has_digits_flag = digits > 0                            # есть ли цифры
    letters = sum(c.isalpha() for c in name)                # количество букв
    entropy = shannon_entropy(name) if length > 0 else 0    # энтропия
    has_dash_flag = "-" in name                             # есть ли дефис
    dash_count = name.count("-")                            # количество дефисов
    digit_ratio_value = digit_ratio(name)                   # доля цифр
    dot_count = domain.count(".")                           # количество точек
    vowel_ratio_value = vowel_ratio(name)                   # доля гласных

    return [
        length,
        digits,
        int(has_digits_flag),
        letters,
        entropy,
        int(has_dash_flag),
        dash_count,
        digit_ratio_value,
        dot_count,
        vowel_ratio_value,
    ]

In [6]:
from sklearn.model_selection import train_test_split

data = pd.read_csv("dga.csv")

train, test = train_test_split(data, test_size=0.25, random_state=42)

In [7]:
len(train), len(test)

(13289842, 4429948)

In [9]:
X_train = np.array([
    extract_features(str(d))
    for d in tqdm(train["domain"], desc="Extracting train features") # прогресс-бар
])

y_train = train["label"].values

X_test = np.array([
    extract_features(str(d))
    for d in tqdm(test["domain"], desc="Extracting test features")
])

y_test = test["label"].values

Extracting test features: 100%|██████████| 4429948/4429948 [01:22<00:00, 53944.26it/s]


In [10]:
# from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

# # Разделение на обучающую и валидационную выборки, чтоб оценить качество модели
# X_tr, X_val, y_tr, y_val = train_test_split(
#     X_train,
#     y_train,
#     test_size=0.2,
#     random_state=42,
#     stratify=y_train
# )

In [11]:
model = make_pipeline(
    StandardScaler(),
    LogisticRegression( # используем простую модель логрега
        max_iter=100,
        random_state=0
    )
)

In [12]:
model.fit(X_train, y_train) # обучаем модель

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(random_state=0))])

In [13]:
y_test_pred = model.predict(X_test) # предсказываем на валидационной выборке

f1 = f1_score(y_test, y_test_pred) # вычисляем f1 скор
# он показывает некую оценку качества модели учитывая точность и полноту
# f1 = 2 * (precision * recall) / (precision + recall)
print(f1)

0.7490576158753428


Давайте посмотрим и на остальные показатели, для этого есть функция classification_report

In [14]:
print(classification_report(y_test, y_test_pred, digits=4))

              precision    recall  f1-score   support

           0     0.7909    0.8286    0.8093   2458744
           1     0.7727    0.7268    0.7491   1971204

    accuracy                         0.7833   4429948
   macro avg     0.7818    0.7777    0.7792   4429948
weighted avg     0.7828    0.7833    0.7825   4429948



classification_report вывел значения **precision**, **recall**, **F1-score** и **support** для каждого класса:

- **0** — нормальные домены  
- **1** — DGA-домены  

Также выводятся:

- **accuracy** — общая доля правильных предсказаний  

- **macro avg** — среднее арифметическое метрик по классам  

- **weighted avg** — среднее метрик, взвешенное по количеству объектов каждого класса  


In [ ]:
# Так можно сделать прогноз для отправки на Kaggle

# test["label"] = model.predict(Xtest).astype(int)
# test[["id", "label"]].to_csv("submission.csv", index=False) # сохраняем предсказания на тестовой выборке

Далее можно загрузить файл `submission.csv` на страницу соревнования Kaggle  
[**DGA Domain Detection Challenge**](https://www.kaggle.com/competitions/dga-domain-detection-challenge).

Kaggle автоматически оценит ваше решение на **скрытой тестовой выборке** (private leaderboard)  
и вернёт итоговый **score**, отражающий качество предсказаний по метрике соревнования.
